In [1]:
##Cormac McHale

In [2]:
##Training the Neural Network

In [3]:
import gzip
import sklearn.preprocessing as pre
import keras as kr
import numpy as np

Using TensorFlow backend.


In [4]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=1000, activation='sigmoid', input_dim=784))
model.add(kr.layers.Dense(units=1000, activation='sigmoid'))
# Add a 10 neuron output layer for each digit??
model.add(kr.layers.Dense(units=10, activation='sigmoid'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
imagesFile  = open('MNIST/train-images.idx3-ubyte', 'rb')

In [6]:
labelsFile = open('MNIST/train-labels.idx1-ubyte', 'rb')
images = imagesFile.read()
labels = labelsFile.read()

In [10]:
images = ~np.array(list(images[16:])).reshape(60000, 28, 28).astype(np.uint8)/255-1.0
labels =  np.array(list(labels[ 8:])).astype(np.uint8)

In [11]:
inputs = images.reshape(60000, 784)

In [12]:
encoder = pre.LabelBinarizer()
encoder.fit(labels)
outputs = encoder.transform(labels)
print(inputs[0])

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


In [15]:
##for a later consideration
model.fit(inputs, outputs, epochs=3, batch_size=50)

Epoch 1/3
60000/60000 [==============================] - 66s 1ms/step - loss: 0.1674 - accuracy: 0.9485
Epoch 2/3
60000/60000 [==============================] - 71s 1ms/step - loss: 0.0986 - accuracy: 0.9693
Epoch 3/3
60000/60000 [==============================] - 72s 1ms/step - loss: 0.0686 - accuracy: 0.9779


In [ ]:
##need to then use the test images on the Neural Network...